In [ ]:
#!/usr/bin/env python3
import os, platform, subprocess, sys, requests, time

In [ ]:
"""
Finding and importing provided libraries regardless of system and platform type
"""
## find and import libraries relative to this file's location (regardless of script or notebook)
## platfrom specific naming already handled within the lib directory's __init__ file
if '__file__' not in globals():
    sys.path.append(os.path.dirname(os.path.abspath('').split('transactions')[0]))
else:
    sys.path.append(os.path.dirname(__file__).split('transactions')[0])

from lib.encoder  import encode_tx, encode_script
from lib.helper   import decode_address
from lib.hash     import hash160, hash256, sha256
from lib.rpc      import RpcSocket
from lib.rpcauth  import rpcauth
from lib.sign     import sign_tx

In [ ]:
'''
Step 0.0: setup our RPC socket like we did in the previous example
        If you have not done the previous example, now would be the time.
        Please copy in the variables you used (located conveniently at the end of the previous example)
'''

nodeIP = "127.0.0.1"
nodePort = "18444"
username = "user"
password = "password"
wallet = "test2"

rpc = RpcSocket({ 'wallet': wallet,
                  'username': username,
                  'password': password,
                  'url' : nodeIP,
                  'port': nodePort})
assert rpc.check()

In [ ]:
'''
Step 0.1: make sure you have cash. If you are broke, see previous example on how to get funds
'''
if float(rpc.call("getbalance")) == 0:
    raise Exception("get funds OR check if funds are still in a pending (ie. unconfirmed) state")
else:
    print("You have a balance of:", rpc.call("getbalance"), "BTC")

In [ ]:
'''
First, we will lookup an existing utxo, and use that to fund our transaction.
'''
utxo = rpc.get_utxo(0)
print(utxo)

In [ ]:
## We will also grab a new receiving address,
## and lock the funds to this address.
recv = rpc.get_recv()
print(recv)

In [ ]:
## Replace this default preimage with your own secret.
secret_preimage = 'weareallsatoshi'

## Convert the secret to bytes, then hash using hash160 function.
secret_bytes = secret_preimage.encode('utf8').hex()
secret_hash  = hash160(secret_bytes).hex()

In [ ]:
## This is where we specify the version number for the program interpreter. 
## We'll be using version 0.
script_version = 0

## Here is the locking script that we will be using. We are going to
## require the redeemer to reveral the secret, along with their public
## key for the receipt address, and matching signature.
script_words = [
    'OP_HASH160', secret_hash, 'OP_EQUALVERIFY', 
    'OP_DUP', 'OP_HASH160', hash160(recv['pub_key']).hex(), 'OP_EQUALVERIFY', 
    'OP_CHECKSIG',
]

In [ ]:
## This is the hex-encoded format of the script. We will present this when 
## we unlock and spend the output. It should match the pre-image used for 
## making the script hash.
redeem_script = encode_script(script_words, prepend_len=False).hex()

## We hash the script using sha256, then provide a version number
## along with the hash. This will lock the transaction output to 
## accept only the program script which matches the hash.
script_hash = sha256(redeem_script).hex()

## Calculate the value of the transaction output, minus fees.
locking_tx_value = utxo['value'] - 1000

In [ ]:
## The initial locking transaction. This spends the utxo from our funding 
## transaction, and moves the funds to the utxo for our witness program.
locking_tx = {
    'version': 1,
    'vin': [{
        'txid': utxo['txid'],
        'vout': utxo['vout'],
        'script_sig': [],
        'sequence': 0xFFFFFFFF
    }],
    'vout': [{
        'value': locking_tx_value,
        'script_pubkey': [ script_version, script_hash ]
    }],
    'locktime': 0
}

In [ ]:
## Encode the transaction into raw hex,
## and calculate the transaction ID
locking_hex  = encode_tx(locking_tx)
locking_txid = hash256(bytes.fromhex(locking_hex))[::-1].hex()

In [ ]:
## Sign the transaction using our key-pair from the utxo.
locking_sig = sign_tx(
  locking_tx,               # The transaction.
  0,                        # The input being signed.
  utxo['value'],            # The value of the utxo being spent.
  utxo['pubkey_hash'],
  utxo['priv_key']          # The private key to the utxo pubkey hash.
)

In [ ]:
## Add the signature and public key to the transaction.
locking_tx['vin'][0]['witness'] = [ locking_sig, utxo['pub_key'] ]

In [ ]:
print(f'''
# Pay-to-Witness-Script-Hash Example

Locking Txid:
{locking_txid}

Redeem Script:
{redeem_script}

Locking Tx:
{encode_tx(locking_tx)}
''')

In [ ]:
## send it off :D
locking_tx_id = rpc.transact(encode_tx(locking_tx))
print(locking_tx_id)

In [ ]:
## Bech32 addresses will decode into a script version and pubkey hash.
script_version, pubkey_hash = decode_address(recv['address'])

In [ ]:
## This transaction will redeem the previous utxo by providing the secret 
## pre-image, plus the public key and signature, plus the witness program. 
## Once the transaction is confirmed, your wallet software should recognize 
## this utxo as spendable.
redeem_tx = {
    'version': 1,
    'vin': [{
        'txid': locking_txid,
        'vout': 0,
        'script_sig': [],
        'sequence': 0xFFFFFFFF
    }],
    'vout': [{
        'value': locking_tx_value - 1000,
        'script_pubkey': [ script_version, pubkey_hash ]
    }],
    'locktime':0
}

redeem_sig = sign_tx(
  redeem_tx,
  0,
  locking_tx_value,
  redeem_script,
  recv['priv_key']
)

In [ ]:
redeem_tx['vin'][0]['witness'] = [ redeem_sig, recv['pub_key'], secret_bytes, redeem_script ]

print(f'Unlocking Tx:\n{encode_tx(redeem_tx)}')

In [ ]:
## redeem it
redeem_tx_id = rpc.transact(encode_tx(redeem_tx))
print(redeem_tx_id)

Bonus: Fun with mempool

https://mempool.space/ is a site that you can look up the state of various popular blockchains

DOES NOT WORK WITH REGTEST!!!!!


In [ ]:
'''
get the info on which chain we are on
'''
chain = rpc.getChain()['chain']
if chain == 'main':
    chain = ''
elif chain == 'test':
    chain = 'testnet'
else:
    chain = 'regtest'

In [ ]:
'''
The original utxo that funded the transaction can be found here
'''
if chain != 'regtest':
    url = "https://mempool.space/"+chain+"/tx/" + utxo['txid']
    print("https://mempool.space/"+chain+"/tx/" + utxo['txid'])

In [ ]:
'''
The original utxo that funded the transaction can be found here
We have obtained several fields about this utxo.
First, let's find this transaction on testnet mem pool, using its REST api. And print some stats.
'''
if chain != 'regtest':
    print("https://mempool.space/"+chain+"/tx/" + utxo['txid']+ "\n")
    api_url = "https://mempool.space/"+chain+"/api/tx/" + utxo['txid']
    response = requests.get(api_url)
    print(response.json())

In [ ]:
'''
The mem pool REST api can also tell us the status of this transaction
'''
if chain != 'regtest':
    api_url = "https://mempool.space/"+chain+"/api/tx/" + utxo['txid'] + "/status"
    response = requests.get(api_url)
    print(response.json())

In [ ]:
'''
Next, let's find the original utxo address on testnet mem pool.
'''
if chain != 'regtest':
    api_url = "https://mempool.space/"+chain+"/api/address/" + utxo['address']
    response = requests.get(api_url)
    print(response.json())

In [ ]:
'''
We can also get the transaction history for this address
'''
if chain != 'regtest':
    api_url = "https://mempool.space/"+chain+"/api/address/" + utxo['address'] + '/txs'
    response = requests.get(api_url)
    print(response.json())

In [ ]:
'''
We can find the recv address on the mem pool as well (sleep to give transaction a chance to propogate!)
'''
if chain != 'regtest':
    print("https://mempool.space/"+chain+"/address/" + recv['address'] +"\n")
    api_url = "https://mempool.space/"+chain+"/api/address/" + recv['address']
    response = requests.get(api_url)
    print(response.json())

In [ ]:
'''
And information on the locking transaction
Delay is added to give time for the node to propagate the transaction
'''
if chain != 'regtest':
    #time.sleep(20) 
    print("https://mempool.space/"+chain+"/tx/" + locking_tx_id + "\n")
    api_url = "https://mempool.space/"+chain+"/api/tx/" + locking_tx_id
    response = requests.get(api_url)
    print(response.json())

In [ ]:
'''
And redeem transaction
''' 
if chain != 'regtest':
    api_url = "https://mempool.space/"+chain+"/api/tx/" + redeem_tx_id
    response = requests.get(api_url)
    print(response.json())

In [ ]:
'''
And we can confirm if it has made it into a block or not
'''
if chain != 'regtest':
    api_transact_url = "https://mempool.space/"+chain+"/api/tx/" + redeem_tx_id + "/status"
    response = requests.get(api_transact_url)
    print(response.json())